Import necessary libraries

In [246]:
import pandas as pd
import numpy as np
import random

Read excel file that contains populations and information of whether they are affected city or distribution city

In [247]:
df = pd.read_excel("../Datasets/cities.xlsx")

Show first 15 cities in the dataframe ( Descending population)

In [248]:
df.head(15)

,Name,Affected,Dist,Population
0,İstanbul,0,1,15462452
1,Ankara,0,1,5663322
2,İzmir,0,1,4394694
3,Bursa,0,1,3101833
4,Antalya,0,1,2548308
5,Adana,1,0,2258718
6,Konya,0,1,2250020
7,Şanlıurfa,1,0,2115256
8,Gaziantep,1,0,2101157
9,Kocaeli,0,1,1997258


Create arrays to store distribution cities and affected cities' names

In [249]:
dist_centers = df[df["Dist"]==1]["Name"].str.title().to_list()
affected_cities = df[df["Affected"]==1]["Name"].str.title().to_list()

Names of distribution cities

In [250]:
dist_centers

['İstanbul',
 'Ankara',
 'İzmir',
 'Bursa',
 'Antalya',
 'Konya',
 'Kocaeli',
 'Mersin',
 'Manisa',
 'Kayseri']

Names of the affected cities

In [251]:
affected_cities

['Adana',
 'Şanlıurfa',
 'Gaziantep',
 'Diyarbakır',
 'Hatay',
 'Kahramanmaraş',
 'Malatya',
 'Adıyaman',
 'Elazığ',
 'Osmaniye',
 'Kilis']

Create distances matrix

In [252]:
distances = pd.read_excel("../Datasets/ilmesafe.xlsx")

In [253]:
distances

,Name,Adana,Adıyaman,Afyonkarahisar,Ağrı,Amasya,Ankara,Antalya,Artvin,Aydın,...,Batman,Şırnak,Bartın,Ardahan,Iğdır,Yalova,Karabük,Kilis,Osmaniye,Düzce
0,Adana,NaN,337.0,578.0,973.0,603.0,492.0,547.0,1001.0,880.0,...,628.0,712.0,784.0,1037.0,1077.0,905.0,714.0,251.0,89.0,735.0
1,Adıyaman,337.0,NaN,915.0,642.0,627.0,732.0,884.0,715.0,1217.0,...,297.0,464.0,1016.0,751.0,746.0,1137.0,946.0,210.0,244.0,966.0
2,Afyonkarahisar,578.0,915.0,NaN,1320.0,596.0,255.0,287.0,1224.0,340.0,...,1207.0,1290.0,518.0,1361.0,1464.0,346.0,449.0,830.0,667.0,371.0
3,Ağrı,973.0,642.0,1320.0,NaN,740.0,1055.0,1424.0,364.0,1634.0,...,365.0,424.0,1171.0,305.0,144.0,1363.0,1106.0,812.0,880.0,1192.0
4,Amasya,603.0,627.0,596.0,740.0,NaN,331.0,823.0,679.0,935.0,...,805.0,998.0,436.0,781.0,884.0,629.0,371.0,640.0,603.0,458.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,Yalova,905.0,1137.0,346.0,1363.0,629.0,407.0,610.0,1238.0,507.0,...,1403.0,1549.0,373.0,1347.0,1507.0,NaN,351.0,1116.0,994.0,171.0
77,Karabük,714.0,946.0,449.0,1106.0,371.0,216.0,735.0,958.0,788.0,...,1174.0,1358.0,86.0,1067.0,1250.0,351.0,NaN,925.0,803.0,180.0
78,Kilis,251.0,210.0,830.0,812.0,640.0,711.0,798.0,885.0,1131.0,...,467.0,550.0,995.0,920.0,915.0,1116.0,925.0,NaN,159.0,945.0
79,Osmaniye,89.0,244.0,667.0,880.0,603.0,581.0,636.0,909.0,969.0,...,535.0,619.0,873.0,944.0,984.0,994.0,803.0,159.0,NaN,823.0


In [254]:
# Create a new dataframe
new_df = pd.DataFrame(columns=distances.columns)

# Iterate over each row in the original dataframe
for index, row in distances.iterrows():
    if row['Name'] in dist_centers:
        # Append the row to the new dataframe
        new_df = pd.concat([new_df, row.to_frame().T])

# Reset the index of the new dataframe
new_df.reset_index(drop=True, inplace=True)

Distances to affected cities by each distribution city

In [255]:
distance_df = new_df[affected_cities]

In [256]:
distance_df

,Adana,Şanlıurfa,Gaziantep,Diyarbakır,Hatay,Kahramanmaraş,Malatya,Adıyaman,Elazığ,Osmaniye,Kilis
0,492.0,786.0,647.0,898.0,688.0,572.0,652.0,732.0,749.0,581.0,711.0
1,547.0,902.0,763.0,1084.0,743.0,744.0,953.0,884.0,1051.0,636.0,798.0
2,856.0,1158.0,1020.0,1270.0,1052.0,944.0,1024.0,1104.0,1121.0,945.0,1083.0
3,70.0,424.0,286.0,606.0,265.0,267.0,466.0,407.0,564.0,159.0,321.0
4,951.0,1237.0,1098.0,1349.0,1147.0,1023.0,1103.0,1183.0,1200.0,1040.0,1162.0
5,907.0,1261.0,1123.0,1436.0,1102.0,1104.0,1190.0,1244.0,1287.0,995.0,1158.0
6,335.0,468.0,330.0,581.0,428.0,255.0,335.0,414.0,432.0,356.0,393.0
7,840.0,1127.0,988.0,1239.0,1036.0,913.0,993.0,1072.0,1090.0,929.0,1051.0
8,357.0,712.0,573.0,887.0,553.0,555.0,641.0,695.0,738.0,446.0,609.0
9,886.0,1240.0,1102.0,1415.0,1081.0,1083.0,1169.0,1223.0,1266.0,974.0,1137.0


In [257]:
distance_matrix = np.array(distance_df)

In [258]:
MAX_HELICOPTER_CAPACITY = 20
HELICOPTER_SPEED = 200
MAX_TRUCK_CAPACITY = 50
TRUCK_SPEED = 100
HELICOPTER_NUMBER = 10
TRUCK_NUMBER = 500
NUM_DISTRIBUTION_CENTERS = 10
NUM_AFFECTED_CITIES = 11

In [259]:
supplies = [1546.2452, 566.3322, 439.4694, 310.1833, 254.8308,
            225.002, 199.7258, 186.8757, 145.0616, 142.1455]

# Demand of each demand city
demands = [1000.0, 922.0, 895.0, 859.0, 838.0, 824.0,
           803.0, 783.0, 778.0, 681.0, 632.0]

In [260]:
supplies_new = [i*(9/4) for i in supplies]
supplies = supplies_new

In [261]:
supplies

[3479.0517,
 1274.2474499999998,
 988.80615,
 697.912425,
 573.3693000000001,
 506.2545,
 449.38304999999997,
 420.470325,
 326.3886,
 319.827375]

In [262]:
max_helicopter = [(2.5/10* i) for i in supplies]

In [263]:
max_truck = [7.5/10* i for i in supplies]

# Genetic Algorithm

In [264]:
def create_population(size, max_truck, max_helicopter, supplies, demands):
    population = []
    for _ in range(size):
        # Copy maximum capacities
        remaining_truck = max_truck.copy()
        remaining_helicopter = max_helicopter.copy()
        
        # Shuffle the order of indices for i and j
        indices_i = list(range(10))
        indices_j = list(range(11))
        
        # Shuffle the order of indices again for i and j
        random.shuffle(indices_i)
        random.shuffle(indices_j)

        # Create a 2D list for truck transfers with random values
        truck_transfers = np.zeros((10, 11), dtype=int)
        for i in indices_i:
            for j in indices_j:
                # Maximum transfer allowed is the lesser of remaining supply, demand, and remaining truck capacity
                max_transfer = min(supplies[i], demands[j], remaining_truck[i])  
                truck_transfer = np.random.randint(0, max_transfer + 1)
                truck_transfers[i][j] = truck_transfer
                # Decrease remaining truck capacity
                remaining_truck[i] -= truck_transfer
                
        random.shuffle(indices_i)
        random.shuffle(indices_j)
        
        # Create a 2D list for helicopter transfers based on truck transfers
        helicopter_transfers = np.zeros((10, 11), dtype=int)
        for i in indices_i:
            for j in indices_j:
                # Maximum transfer allowed is the lesser of remaining supply, remaining demand, and remaining helicopter capacity
                remaining_supply = supplies[i] - truck_transfers[i][j]
                remaining_demand = demands[j] - truck_transfers[i][j]
                max_transfer = min(remaining_supply, remaining_demand, remaining_helicopter[i])  
                helicopter_transfer = np.random.randint(0, max_transfer + 1)  # Generate random helicopter transfer
                helicopter_transfers[i][j] = helicopter_transfer
                # Decrease remaining helicopter capacity
                remaining_helicopter[i] -= helicopter_transfer
        individual = [
            truck_transfers,
            helicopter_transfers
        ]
        population.append(individual)
    return population

In [265]:
def fitness_function(individual):
    truck_transfers = np.array(individual[0])
    helicopter_transfers = np.array(individual[1])
    total_cost = np.sum(distance_matrix * (helicopter_transfers / HELICOPTER_SPEED))\
    + np.sum(distance_matrix * (truck_transfers / TRUCK_SPEED))
    penalty = 0
    for i in range(len(supplies)):
        if (sum(truck_transfers[i]) + sum(helicopter_transfers[i])) > supplies[i] * 1.2:
            penalty +=1
        elif sum(truck_transfers[i]) > max_truck[i] *1.2\
        or sum(helicopter_transfers[i]) > max_helicopter[i] *1.2:
            penalty +=1
    total_cost += 1e6 *penalty
    return -total_cost

In [266]:
def selection(population, num_parents, fitnesses):
    # calculate total fitness of all individuals
    total_fitness = sum(fitnesses)
    # calculate relative fitness of each individual
    rel_fitness = [f/total_fitness for f in fitnesses]
    # generate probability intervals for each individual
    probs = [sum(rel_fitness[:i+1]) for i in range(len(rel_fitness))]
    # draw new population
    new_population = []
    for n in range(num_parents):
        r = random.random()
        for (i, individual) in enumerate(population):
            if r <= probs[i]:
                new_population.append(individual)
                break
    return new_population

In [267]:
def uniform_crossover(parent1, parent2):
    # Define a crossover rate
    crossover_rate = 0.5

    # Perform uniform crossover
    child1_truck = []
    child2_truck = []
    child1_heli = []
    child2_heli = []

    # Crossover for truck transfers
    for i in range(len(parent1[0])):
        if np.random.random() < crossover_rate:
            child1_truck.append(parent1[0][i])
            child2_truck.append(parent2[0][i])
        else:
            child1_truck.append(parent2[0][i])
            child2_truck.append(parent1[0][i])

    # Crossover for helicopter transfers
    for i in range(len(parent1[1])):
        if np.random.random() < crossover_rate:
            child1_heli.append(parent1[1][i])
            child2_heli.append(parent2[1][i])
        else:
            child1_heli.append(parent2[1][i])
            child2_heli.append(parent1[1][i])
    
    return [np.array(child1_truck), np.array(child1_heli)], [np.array(child2_truck), np.array(child2_heli)]

In [268]:
def mutation(individual, mutation_rate, max_truck, max_helicopter, supplies, demands):
    truck_transfers, helicopter_transfers = individual

    for i in range(10):
        for j in range(11):
            if np.random.rand() < mutation_rate:
                    truck_transfers[i][j] += np.random.randint(0,3)
                    helicopter_transfers[i][j] += np.random.randint(0,3)

    return [truck_transfers, helicopter_transfers]

In [269]:
def genetic_algorithm(population_size=100, num_generations=200, mutation_rate=0.01):
    # Create initial population
    population = create_population(population_size, max_truck, max_helicopter, supplies, demands)
    
    # Evaluate the population
    fitness_values = [fitness_function(individual) for individual in population]

    # Iterate through each generation
    for generation in range(num_generations):

        # Create a new population
        new_population = []

        while len(new_population) < population_size:
            # Select two parents
            parent1, parent2 = selection(population, 2, fitness_values)
            
            # Create two children by crossover
            child1, child2 = uniform_crossover(parent1, parent2)

            # Add the children to the new population
            new_population += [child1, child2]

        # Apply mutation
        new_population = [mutation(individual, mutation_rate, max_truck, max_helicopter, supplies, demands) for individual in new_population]

        # Replace the old population with the new population
        population = new_population

        # Evaluate the new population
        fitness_values = [fitness_function(individual) for individual in population]
        
    # Return the individual with the best fitness value
    best_fitness_index = np.argmax(fitness_values)
    best_individual = population[best_fitness_index]

    return best_individual



population_size = 200
num_generations = 200
mutation_rate = 0.01

best_individual = genetic_algorithm(population_size, num_generations, mutation_rate)

total cost:64054.025
total cost:62334.505000000005
total cost:67304.58499999998
total cost:61870.44499999999
total cost:60119.98000000001
total cost:62706.22000000002
total cost:57180.355
total cost:60729.75000000002
total cost:61336.70000000001
total cost:62860.44500000001
total cost:60283.000000000015
total cost:59535.25500000002
total cost:67341.87000000004
total cost:63854.07
total cost:65375.55499999998
total cost:66500.04499999998
total cost:66796.285
total cost:62996.729999999996
total cost:63990.399999999994
total cost:64330.37000000002
total cost:68731.95999999999
total cost:66453.31499999999
total cost:66206.37999999999
total cost:61640.97000000003
total cost:66792.155
total cost:63968.149999999994
total cost:61842.880000000005
total cost:55350.75
total cost:65827.30500000002
total cost:56861.30499999999
total cost:60893.64
total cost:62016.405
total cost:65477.56500000001
total cost:64293.325000000055
total cost:67200.725
total cost:62753.92
total cost:60797.45499999998
tota

total cost:63025.385
total cost:65727.635
total cost:64344.76000000004
total cost:65955.52500000001
total cost:65777.98
total cost:65775.16
total cost:60676.50500000001
total cost:65104.34500000001
total cost:62474.030000000035
total cost:61061.72499999998
total cost:59584.840000000004
total cost:59967.225000000006
total cost:63397.655000000006
total cost:60474.99500000001
total cost:64648.205
total cost:63862.99999999999
total cost:63712.24999999999
total cost:64753.72
total cost:66019.23000000001
total cost:64355.120000000024
total cost:66846.01500000004
total cost:62427.045
total cost:63165.67
total cost:58434.674999999996
total cost:59236.890000000014
total cost:60934.835
total cost:59689.78500000002
total cost:59482.44499999998
total cost:64678.15999999999
total cost:61463.01
total cost:59431.54500000003
total cost:57176.704999999994
total cost:64946.345000000016
total cost:61617.924999999996
total cost:61663.87500000001
total cost:58733.740000000005
total cost:63220.225000000006


total cost:64686.665
total cost:64289.92
total cost:61717.289999999986
total cost:63831.235000000015
total cost:62777.11
total cost:59086.389999999985
total cost:60493.35
total cost:61245.29500000003
total cost:62333.22999999997
total cost:62556.98500000001
total cost:62075.62000000001
total cost:64070.599999999984
total cost:64684.93999999999
total cost:63652.725000000006
total cost:61322.030000000006
total cost:59332.424999999996
total cost:63610.555
total cost:66395.51000000002
total cost:62984.485000000015
total cost:59772.67500000002
total cost:64544.645
total cost:63021.510000000024
total cost:62069.21
total cost:63606.82500000001
total cost:65045.46499999999
total cost:63064.6
total cost:63167.665
total cost:59733.96500000001
total cost:60469.79500000001
total cost:65150.28000000001
total cost:62792.28999999998
total cost:64884.06500000001
total cost:63218.26499999999
total cost:61905.685000000005
total cost:62124.18000000001
total cost:61572.000000000015
total cost:62465.374999

total cost:60239.360000000015
total cost:62708.19499999997
total cost:62521.705
total cost:64303.235
total cost:64059.12500000003
total cost:60807.854999999996
total cost:61876.104999999996
total cost:64049.695000000014
total cost:61820.30499999997
total cost:65556.43499999997
total cost:63771.130000000005
total cost:59512.41499999997
total cost:60797.04000000004
total cost:66111.255
total cost:65237.77999999999
total cost:63771.64000000002
total cost:63154.63500000001
total cost:65139.79500000001
total cost:62682.980000000025
total cost:62595.96999999997
total cost:62444.21000000001
total cost:64719.144999999975
total cost:61931.53000000001
total cost:64553.66000000001
total cost:63602.58
total cost:62096.67000000001
total cost:62060.54
total cost:64326.38500000002
total cost:66983.51499999998
total cost:65749.23999999998
total cost:64041.94000000003
total cost:62326.39500000001
total cost:64239.18000000001
total cost:62632.14500000002
total cost:61547.055000000015
total cost:63964.54

total cost:63134.475
total cost:65304.29000000004
total cost:63190.645000000004
total cost:62628.29
total cost:61404.085
total cost:61038.92000000001
total cost:61534.445000000014
total cost:64607.100000000006
total cost:63426.58
total cost:64484.62000000001
total cost:61106.900000000016
total cost:63986.56499999998
total cost:64230.015000000014
total cost:63558.300000000025
total cost:64269.74500000002
total cost:65788.46500000001
total cost:63121.765
total cost:61714.05500000001
total cost:63435.9
total cost:63270.329999999994
total cost:63712.71
total cost:62202.55000000002
total cost:65076.31499999999
total cost:64527.65500000001
total cost:65644.92000000003
total cost:63326.35999999999
total cost:64590.285
total cost:64758.19
total cost:59696.76000000001
total cost:61277.60499999999
total cost:66126.30500000002
total cost:62313.21000000001
total cost:65224.8
total cost:61964.07999999999
total cost:65273.515000000014
total cost:63333.04000000001
total cost:62048.075
total cost:6263

...

total cost:69010.09500000003
total cost:70444.59500000003
total cost:70482.21000000006
total cost:70682.66000000002
total cost:68855.50000000003
total cost:69460.80000000002
total cost:69132.15500000003
total cost:69131.95500000002
total cost:69242.40000000001
total cost:70738.77500000004
total cost:69063.34500000004
total cost:69234.07500000003
total cost:69576.87500000001
total cost:69086.44000000002
total cost:69408.80000000003
total cost:69297.87
total cost:70432.31500000005
total cost:69128.75000000003
total cost:69806.11500000002
total cost:69672.95500000003
total cost:69262.83500000006
total cost:70911.41000000002
total cost:69890.0
total cost:69363.99
total cost:68983.38
total cost:69783.60500000003
total cost:69547.515
total cost:69031.62500000004
total cost:69784.31500000002
total cost:69311.61000000003
total cost:68343.71500000001
total cost:69911.00000000003
total cost:69349.90000000004
total cost:70236.40999999999
total cost:70667.47500000002
total cost:70175.805
total cos

total cost:70947.865
total cost:69403.12500000003
total cost:69497.975
total cost:70979.22500000003
total cost:69076.59000000004
total cost:69258.29500000004
total cost:69721.69500000005
total cost:69832.44500000004
total cost:70464.63000000005
total cost:70344.21000000002
total cost:70317.26500000003
total cost:69032.27500000001
total cost:70395.70500000002
total cost:70738.35500000004
total cost:69683.76500000001
total cost:69580.04500000001
total cost:70340.34500000002
total cost:69284.88000000002
total cost:68789.95000000001
total cost:69890.69000000005
total cost:69753.27000000005
total cost:69536.79500000001
total cost:69183.87500000006
total cost:69537.33000000005
total cost:69882.47000000003
total cost:70391.04500000001
total cost:69174.35000000003
total cost:70275.66500000004
total cost:68748.73000000001
total cost:69213.91500000004
total cost:70441.07000000004
total cost:69808.39500000002
total cost:70793.57499999998
total cost:69182.21000000002
total cost:69994.72499999999
t

total cost:70803.09000000001
total cost:70911.44000000003
total cost:69866.73500000004
total cost:68987.18000000001
total cost:68978.695
total cost:69629.72000000002
total cost:69029.28000000003
total cost:70400.32000000004
total cost:70288.31000000001
total cost:68897.39500000002
total cost:69211.16500000004
total cost:69168.29500000004
total cost:68583.00000000003
total cost:70462.46000000005
total cost:69214.42
total cost:69792.16000000003
total cost:70634.26999999999
total cost:69410.18500000006
total cost:69603.935
total cost:69608.95000000001
total cost:71348.61500000002
total cost:69930.79000000002
total cost:70171.46000000002
total cost:69765.56000000003
total cost:69270.25500000003
total cost:70423.08500000002
total cost:70754.76
total cost:69732.84000000001
total cost:69364.80000000005
total cost:70512.10000000002
total cost:69741.34500000002
total cost:70136.46500000001
total cost:70467.94000000006
total cost:69614.29000000001
total cost:68651.95500000003
total cost:69755.90

total cost:69582.52500000002
total cost:69682.81999999998
total cost:70199.83499999999
total cost:69623.50500000002
total cost:69167.82500000001
total cost:70319.76000000001
total cost:69159.49500000001
total cost:70481.62500000003
total cost:68745.97000000002
total cost:69733.76500000003
total cost:69505.50000000004
total cost:69063.69000000005
total cost:70174.92000000003
total cost:71067.36000000003
total cost:68923.67500000002
total cost:71340.86500000002
total cost:70007.75000000003
total cost:70458.215
total cost:70187.00500000005
total cost:71166.24
total cost:70013.02500000004
total cost:69368.17000000004
total cost:70289.005
total cost:69822.80000000002
total cost:69287.59500000004
total cost:71796.375
total cost:69876.03500000003
total cost:70284.50500000002
total cost:69494.14000000004
total cost:70854.5
total cost:69784.395
total cost:68259.52000000003
total cost:69636.34500000006
total cost:70186.765
total cost:70184.205
total cost:69688.63500000001
total cost:69596.595
to

total cost:69443.61
total cost:69710.57500000001
total cost:69929.69500000005
total cost:71188.815
total cost:71565.22000000004
total cost:69620.68500000003
total cost:69726.24500000002
total cost:70566.46000000002
total cost:68679.365
total cost:70371.44500000002
total cost:69765.79000000002
total cost:69662.86500000002
total cost:71445.90500000001
total cost:70333.55000000005
total cost:69941.06499999999
total cost:69734.88
total cost:68542.3
total cost:69983.01000000001
total cost:70595.14000000003
total cost:69126.76500000003
total cost:70671.14500000005
total cost:69743.03499999999
total cost:69636.37499999999
total cost:70508.45000000001
total cost:69844.13500000004
total cost:69434.01
total cost:70023.82500000001
total cost:69366.09000000001
total cost:67852.33000000005
total cost:70556.84000000004
total cost:70070.27500000004
total cost:70312.11499999999
total cost:70058.04000000005
total cost:69813.71500000003
total cost:68708.77000000003
total cost:69499.49499999998
total cos

In [270]:
print("Best Individual(Truck):\n", best_individual[0],"\n")
print("Best Individual(Helicopter):\n", best_individual[1])
print("Best Fitness:", -fitness_function(best_individual))

Best Individual(Truck):
 [[  2 640 543 102 313  27 113 244  74  25 509]
 [  0   1   0 750   8   4   7   2   6 196   2]
 [  2 274   1 130   5   5 277   3   1  69   3]
 [  7  14 479  13   2   3   0  22   0   0   4]
 [  1   5   2   3  87   5   3   0  24 315   2]
 [  4 119  45   4   1   4   0   3   4   0 219]
 [  0  30   5   7  41  53   3   6  85   1 124]
 [  4   3   3 231   6   5   2   6  81   9   0]
 [  0  15   2  57   3   5   2 177   1   6   2]
 [  1   7   0   1 111 122   3   6   1   1   6]] 

Best Individual(Helicopter):
 [[  0   2   2   0 787   0   9   5  41   0  32]
 [ 15   0   3   3 233   4  32  20   2  11   9]
 [ 58   3   2   4   2   1 193   0   3   2   4]
 [ 30 102   6  32   2  11  15  11   5   0   3]
 [  7   2  28   1  11  14   0   3  28   3  67]
 [  2  65   3  43   1   4   2   0   5  14   7]
 [ 57   8  58   1   0   5   0   0   3   3   3]
 [  1   2   5   5  11   0   4   2   2   5 105]
 [  2   2   3   2   6  19  33  27   0  15   0]
 [  5   5  13  39  13   5   4   8   7  20   5]]
t

In [271]:
supplies

[3479.0517,
 1274.2474499999998,
 988.80615,
 697.912425,
 573.3693000000001,
 506.2545,
 449.38304999999997,
 420.470325,
 326.3886,
 319.827375]

In [272]:
sum(best_individual[0][0])

2592

In [273]:
sum(best_individual[1][0])

878

In [274]:
max_truck

[2609.288775,
 955.6855874999999,
 741.6046125,
 523.43431875,
 430.02697500000005,
 379.690875,
 337.0372875,
 315.35274375,
 244.79145,
 239.87053125]

In [275]:
max_helicopter

[869.762925,
 318.56186249999996,
 247.2015375,
 174.47810625,
 143.34232500000002,
 126.563625,
 112.34576249999999,
 105.11758125,
 81.59715,
 79.95684375]

# Simulated Annealing

In [276]:
def create_population(size, max_truck, max_helicopter, supplies, demands):
    population = []
    for _ in range(size):
        # Copy maximum capacities
        remaining_truck = max_truck.copy()
        remaining_helicopter = max_helicopter.copy()
        
        # Shuffle the order of indices for i and j
        indices_i = list(range(10))
        indices_j = list(range(11))
        
        # Shuffle the order of indices again for i and j
        random.shuffle(indices_i)
        random.shuffle(indices_j)

        # Create a 2D list for truck transfers with random values
        truck_transfers = np.zeros((10, 11), dtype=int)
        for i in indices_i:
            for j in indices_j:
                # Maximum transfer allowed is the lesser of remaining supply, demand, and remaining truck capacity
                max_transfer = min(supplies[i], demands[j], remaining_truck[i])  
                truck_transfer = np.random.randint(0, max_transfer + 1)
                truck_transfers[i][j] = truck_transfer
                # Decrease remaining truck capacity
                remaining_truck[i] -= truck_transfer
                
        random.shuffle(indices_i)
        random.shuffle(indices_j)
        
        # Create a 2D list for helicopter transfers based on truck transfers
        helicopter_transfers = np.zeros((10, 11), dtype=int)
        for i in indices_i:
            for j in indices_j:
                # Maximum transfer allowed is the lesser of remaining supply, remaining demand, and remaining helicopter capacity
                remaining_supply = supplies[i] - truck_transfers[i][j]
                remaining_demand = demands[j] - truck_transfers[i][j]
                max_transfer = min(remaining_supply, remaining_demand, remaining_helicopter[i])  
                helicopter_transfer = np.random.randint(0, max_transfer + 1)  # Generate random helicopter transfer
                helicopter_transfers[i][j] = helicopter_transfer
                # Decrease remaining helicopter capacity
                remaining_helicopter[i] -= helicopter_transfer
        individual = [
            truck_transfers,
            helicopter_transfers
        ]
        population.append(individual)
    return population

In [277]:
def fitness_function(individual):
    truck_transfers = np.array(individual[0])
    helicopter_transfers = np.array(individual[1])
    total_cost = np.sum(distance_matrix * (helicopter_transfers / HELICOPTER_SPEED))\
    + np.sum(distance_matrix * (truck_transfers / TRUCK_SPEED))
    penalty = 0
    for i in range(len(supplies)):
        if (sum(truck_transfers[i]) + sum(helicopter_transfers[i])) > supplies[i] * 1.2:
            penalty +=1
        elif sum(truck_transfers[i]) > max_truck[i] *1.2\
        or sum(helicopter_transfers[i]) > max_helicopter[i] *1.2:
            penalty +=1
    total_cost += 1e6 *penalty
    return -total_cost

In [278]:
def random_neighbor(individual, max_truck, max_helicopter, supplies, demands):
    truck_transfers, helicopter_transfers = individual
    
    # Shuffle the order of indices for i and j
    indices_i = list(range(10))
    indices_j = list(range(11))
        
    # Shuffle the order of indices again for i and j
    random.shuffle(indices_i)
    random.shuffle(indices_j)
    
    for i in indices_i:
        for j in indices_j:
            if sum(truck_transfers[i]) >= max_truck[i]:
                if random.random() < 0.5:
                    truck_transfers[i][j] -= np.random.randint(0, 60)
                    if truck_transfers[i][j] < 0:
                        truck_transfers[i][j] = 0
            else:
                truck_transfers[i][j] += np.random.randint(-2, 10)
                if truck_transfers[i][j] < 0:
                    truck_transfers[i][j] = 0 
            if sum(helicopter_transfers[i]) >= max_helicopter[i]:
                if random.random() < 0.5:
                    helicopter_transfers[i][j] -= np.random.randint(0, 60)
                    if helicopter_transfers[i][j] < 0:
                        helicopter_transfers[i][j] = 0
            else:
                helicopter_transfers[i][j] += np.random.randint(-2, 10)
                if helicopter_transfers[i][j] < 0:
                        helicopter_transfers[i][j] = 0 

    # Check if the new solution exceeds capacity limits
    for i in range(len(supplies)):
        if (sum(truck_transfers[i]) + sum(helicopter_transfers[i])) > supplies[i] * 1.2 \
                or sum(truck_transfers[i]) > max_truck[i] \
                or sum(helicopter_transfers[i]) > max_helicopter[i]:
            return individual  # Return the original solution if the new solution exceeds capacity limits

    return [truck_transfers, helicopter_transfers]

In [279]:
def acceptance_probability(energy, new_energy, temperature):
    if new_energy < energy:
        return 1.0
    return np.exp((energy - new_energy) / temperature)


In [280]:
def simulated_annealing(initial_solution, max_truck, max_helicopter, supplies, demands, num_iterations, max_temperature):
    current_solution = initial_solution
    best_solution = initial_solution
    current_fitness = fitness_function(current_solution)
    best_fitness = current_fitness

    for iteration in range(num_iterations):
        temperature = max_temperature * (1 - iteration / num_iterations)  # Cooling schedule

        new_solution = random_neighbor(current_solution, max_truck, max_helicopter, supplies, demands)
        new_fitness = fitness_function(new_solution)

        if acceptance_probability(current_fitness, new_fitness, temperature) > random.random():
            current_solution = new_solution
            current_fitness = new_fitness

        if new_fitness > best_fitness:
            best_solution = new_solution
            best_fitness = new_fitness

    return best_solution

In [281]:
population_size = 500
initial_solution = create_population(population_size, max_truck, max_helicopter, supplies, demands)[0]
num_iterations = 1000
max_temperature = 10

best_individual = simulated_annealing(initial_solution, max_truck, max_helicopter, supplies, demands,
                                     num_iterations, max_temperature)

total cost:64604.56000000003
total cost:63131.43000000001
total cost:63942.73500000002
total cost:63848.63999999999
total cost:64362.45499999999
total cost:63902.805
total cost:64014.109999999986
total cost:64911.245
total cost:64151.70000000004
total cost:64885.35999999997
total cost:64156.28
total cost:63770.80500000003
total cost:63017.330000000045
total cost:63832.729999999974
total cost:64230.39499999999
total cost:63641.17000000001
total cost:63815.55000000002
total cost:63662.460000000014
total cost:62985.595
total cost:64296.855000000025
total cost:63776.16500000002
total cost:62892.20499999998
total cost:63318.96000000001
total cost:62532.240000000005
total cost:63138.734999999986
total cost:63836.88499999997
total cost:63686.24500000001
total cost:62884.30500000001
total cost:63753.0
total cost:63104.469999999994
total cost:63357.57
total cost:63168.27499999998
total cost:63214.03499999998
total cost:63138.295
total cost:62517.705
total cost:63186.995000000024
total cost:6305

total cost:61559.36999999997
total cost:61739.49999999999
total cost:61845.68999999999
total cost:61949.23999999999
total cost:62300.645
total cost:61900.11999999999
total cost:61785.42000000003
total cost:62811.54999999999
total cost:61940.055
total cost:62237.53000000001
total cost:62959.00000000001
total cost:63255.755000000005
total cost:62314.21999999998
total cost:63055.23999999999
total cost:62762.564999999995
total cost:63049.345
total cost:63015.43500000003
total cost:61998.41500000004
total cost:62952.88999999998
total cost:63269.32999999999
total cost:62941.82999999999
total cost:63571.64499999999
total cost:63693.34499999998
total cost:63123.22000000001
total cost:62919.92499999997
total cost:63186.74
total cost:63904.05
total cost:63937.104999999996
total cost:64001.69499999999
total cost:63675.11500000002
total cost:63369.95
total cost:64156.53999999999
total cost:64364.325000000004
total cost:64039.27000000001
total cost:63219.789999999986
total cost:63781.06
total cost:

total cost:63178.269999999975
total cost:63584.079999999994
total cost:63088.10000000002
total cost:63168.52999999999
total cost:63124.23500000001
total cost:63326.885
total cost:62961.834999999985
total cost:63171.43500000001
total cost:63299.82999999998
total cost:62947.46999999999
total cost:62965.12499999996
total cost:63591.105000000025
total cost:63724.98
total cost:63544.11500000001
total cost:62886.67
total cost:63039.875000000015
total cost:63405.02500000001
total cost:63418.60500000001
total cost:63455.50500000003
total cost:63088.09
total cost:62746.62000000003
total cost:63306.17500000001
total cost:63253.209999999985
total cost:62982.15500000003
total cost:62949.015
total cost:63944.649999999994
total cost:64165.55500000001
total cost:62970.01
total cost:63364.614999999954
total cost:64210.405
total cost:63641.284999999974
total cost:63807.235000000015
total cost:63346.510000000024
total cost:63358.06000000003
total cost:63849.02000000001
total cost:63988.38999999999
total

In [282]:
print("Best Individual (Truck):\n", best_individual[0], "\n")
print("Best Individual (Helicopter):\n", best_individual[1])
print("Best Fitness:", -fitness_function(best_individual))

Best Individual (Truck):
 [[128 213 156 417 421  68 214 299 130 188 365]
 [ 24 120  47 160  85 149  89  76 103  87   7]
 [192 297  47  43  45  25  12  14  23   0  48]
 [ 29 106  20  54   0   5  70  79 110  17  29]
 [  9  10  49   6   2   7  47 162  20  53  48]
 [ 60  55  21   8  17  19  15  62  23  40  54]
 [ 25   6  34  67  73  26  12   0   0  47  48]
 [ 39   0  40  38  17  49  49  53   0  22  12]
 [  5  39   0  25   0  94  18  12   0  38  17]
 [ 14   5  35   3   7  52  82  17   4   3   7]] 

Best Individual (Helicopter):
 [[ 42   6   0  37 106 373  36  91  74  37  44]
 [ 11  13  33  48  18  50  48   0   8  32  36]
 [ 10  41   7   0  61  18  39  26   0   0  19]
 [  4  38   5  34  24   7  28  16   7   0   9]
 [  9   4   0   0  30  28   0  21   5  32  12]
 [  8   0  21  19   0   1  17   0  22  21   0]
 [ 19   6   0   7  20   0   2  38  14   0   0]
 [  7  37   8   0  15   0   0   3  18  18   0]
 [  0   0   0   0   0   4   1  12  41   2  30]
 [ 26   7   6   9   6   0   7   4  18   0   0]]